In [1]:
import sys
from copy import deepcopy
import time

sys.path.append('../utils')
from pyutils import *

In [2]:
sample = readutf8('sample.txt')
sample_long = readutf8('sample2.txt')
puzzle = readutf8('input.txt')

In [3]:
def parse_input(puzzle_in: str) -> tuple[StrMatrix, list[Point], Point]:
    warehouse_map: StrMatrix = []
    robot_instructions: list[Point] = []
    robot_pos: Point = (-1, -1)
    char_to_point = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}
    part: int = 1
    for line in puzzle_in.split('\n'):
        if line.strip() == '':
            part += 1
            continue
        if part == 1:
            warehouse_map.append(list(line))
            if '@' in line:
                robot_pos = (len(warehouse_map) - 1, line.index('@'))
        elif part == 2:
            robot_instructions.extend(list(map(lambda i: char_to_point[i], line)))
    if robot_pos == (-1, -1):
        raise ValueError('Robot position not found in warehouse map!')
    return (warehouse_map, robot_instructions, robot_pos)

In [4]:
def predict_robot(mat: StrMatrix, instructions: list[Point], robot_start: Point):
    mat = deepcopy(mat)
    def _check_boxes(box_pos: Point) -> Point | None:
        box_future = point_op(add, box_pos, dirpt)
        match (obj := matget(mat, box_future)):
            case '#': ret = None
            case '.': ret = box_pos
            case 'O': ret = _check_boxes(box_future)
            case _: raise ValueError(f'Unexpected character at {box_future!r}: {obj!r}')
        return ret

    robot_pos = robot_start
    for dirpt in instructions:
    # for n, dirpt in enumerate(instructions):
        # time.sleep(0.01)
        # clear_output(wait=True)
        # print(dirpt)
        # print(f'{n:<8} / {len(instructions) - 1:<8}')
        # print(mat_restring(mat))
        robot_future = point_op(add, robot_pos, dirpt)
        match (obj := matget(mat, robot_future)):
            case '#': continue
            case '.':
                matset(mat, robot_pos, '.')
                robot_pos = robot_future
                matset(mat, robot_pos, '@')
            case 'O':
                result = _check_boxes(robot_future)
                if result is not None:
                    matset(mat, robot_pos, '.')
                    robot_pos = robot_future
                    matset(mat, robot_pos, '@')
                    matset(mat, point_op(add, result, dirpt), 'O')

    return mat

In [5]:
def box_gps_coords(mat: StrMatrix) -> list[int]:
    coords: list[int] = []
    for pt, i in mat_iter(mat):
        if i == 'O':
            coords.append((100 * pt[0]) + pt[1])
    return coords

In [11]:
warehouse, robot_moves, robot = parse_input(sample)

In [12]:
print(sum(box_gps_coords(predict_robot(warehouse, robot_moves, robot))))

2028
